In [8]:
import torch
from torch import nn
import copy
import torchvision
from torchvision import transforms
from PIL import Image
from io import BytesIO
import tempfile
import gradio as gr
from gradio.components import Image, Textbox,Label

In [2]:
def load_model(model_path):
    #model = Model()
    #model.load_state_dict(torch.load(model_path, map_location='cpu'))
    checkpoint = torch.load(model_path, map_location=torch.device("cpu"))
    best_model_wts = copy.deepcopy(checkpoint)
    #print(type(checkpoint))
    #print(checkpoint)
    model = torchvision.models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
    model.load_state_dict(best_model_wts)
    return model

In [3]:
model_path = 'model1.pth'
model = load_model(model_path)

In [30]:
class_names = ['artificial','human']

In [33]:
def model_pred(img):
    transform=transforms.Compose([   
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image_tensor=transform(img).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        pred_probs = torch.softmax(model(image_tensor), dim=1)
        #print (max_prob)
        #print (pred_probs)
        pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    #print(preds)
    return pred_labels_and_probs

In [34]:
inputs = gr.Image(type="pil")

#outputs = Textbox(lines=1)
#img=Image.open(inputs)
#img=transforms.ToPILImage(inputs)



interface = gr.Interface(
    fn=model_pred, 
    inputs=inputs, 
    outputs=gr.Label(num_top_classes=2, label="Predictions"), 
    title="Image Classification Prediction",
    description="Provide an image and get the predicted class label.")

interface.launch()

Running on local URL:  http://127.0.0.1:7890

To create a public link, set `share=True` in `launch()`.
